In [1]:
import duckdb
import xml.etree.ElementTree as ET
# Import classifier
from ipynb.fs.full.group_classifier import predict_batch
print("Import successful!!")

2025-06-04 19:16:23.276130: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-04 19:16:26.204611: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749057387.159273  523614 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749057387.514165  523614 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-04 19:16:30.038249: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Import successful
Import successful!!


In [2]:
 # Connect to sql database
con = duckdb.connect(database='../data/database/german-parliament', read_only=False)

In [3]:
def preprocess_speech(speech:tuple) -> tuple[int, list[str]]:
    # We only need the speech it to later identify the origin speech
    speech_id = test_speech[0]
    text = test_speech[5]
    # Parse xml from string will throw ParseError if not parseable
    root = ET.fromstring(text)
    # We only need paragraphs -> this filters unecessary information, such as comments or applause
    paragraphs = root.findall('p')
    paragraphs_text = []
    # Get text of paragraphs
    for p in paragraphs:
        # Use this to filter out speaker information
        if p.attrib.get("klasse") == "redner":
            continue
        else:
            p_text = p.text.strip()
            item = {"paragraphs":p_text}
            paragraphs_text.append(item) #get text and remove potential irrelevant whitespaces

    return (speech_id, paragraphs_text)
    

In [4]:
test_speech = con.sql("select * from speech where id like '%ID%' order by random() limit 1 ").fetchone()

In [8]:
speech_id, paragraphs = preprocess_speech(test_speech)

print(speech_id)
print(paragraphs)

ID1917804000
[{'paragraphs': 'Frau Präsidentin! Meine sehr geehrten Damen und Herren Abgeordnete! Umwelt- und Klimaschutz zahlen sich aus\xa0– das hat heute Morgen unser Finanzminister Olaf Scholz sehr eindrucksvoll dargelegt. Das ist eine der zentralen Botschaften des Bundeshaushaltes für 2021, den die Bundesregierung diese Woche hier in den Deutschen Bundestag einbringt. Sie zahlen sich aus, weil sie gute, gesunde Lebensbedingungen sicherstellen, sie zahlen sich aus, weil sie Standortvorteile sind und Wettbewerbsfähigkeit von morgen sicherstellen, und sie zahlen sich aus, weil sie die Teilhabe aller Menschen in unserem Land sichern und damit auch den Zusammenhalt stärken.'}, {'paragraphs': 'Der Klimaschutz ist daher\xa0– und das ist genau richtig\xa0– auch nicht die alleinige Aufgabe der Umweltministerin. Ich habe immer wieder eingefordert, dass die gesamte Bundesregierung, also alle Ministerien, hier Verantwortung übernehmen. Und dieser Haushaltsentwurf zeigt, dass inzwischen alle M

In [6]:
classified_paragraphs = predict_batch(paragraphs)

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Batch size: 16


In [29]:
def smart_join(tokens):
    print(tokens)
    """
    Joins a list of word tokens into a single string, handling punctuation
    and sub-word prefixes ('##') correctly.

    Args: @todo
    """
    result = []
    # Define punctuation that should not have a preceding space
    no_space_before = {',', '.', '?', '!', ';', ':', ')'}
    
    for i, token in enumerate(tokens):
        # If it's the very first token, a punctuation mark, or a sub-word,
        # don't add a leading space.
        if i > 0 and token not in no_space_before and not token.startswith('##'):
            result.append(' ')
            
        # Append the token itself, removing any '##' prefixes
        result.append(token.replace('##', ''))
        
    return "".join(result)

def extract_groups(index:int, paragraph:list[tuple[str,str]]) -> list[tuple[str, str]]:
    # Maybe adjust this set to tokenizer's special tokens
    SPECIAL = {"[CLS]", "[SEP]", "[PAD]", "[UNK]"}    
    print(f"Processing paragraph {index}")
    # empty list for groups of paragraph
    groups = []
    group_tmp = []
    group_started = False
    entity = "" # This hold the current entity. e.g. EOPOL for B-EOPOL
    for token, label in paragraph:
        # If token is a special token like [CLS] skip it
        if token in SPECIAL:
            continue
        # Check for begin of group
        elif label.startswith("B-"):
            group_started = True
            entity = label[2:]
            if group_tmp:
                groups.append((entity, group_tmp))
                group_tmp = [] # New Group will begin
            # Append new beginning label
            group_tmp.append((token, label))
        # It is checked that 1) we have an inside label and 2) There was a B- label before!
        elif label.startswith("I-") and group_started:  
            # Then we check if the entity matches
            if label[2:] != entity:
                print(f"Current label: {label[2:]} doesn't match beginning label: {entity}")
                # Break current group because of the miss-label
                group_started = False
            else:
            # If all tests hold, we append the token and its label to the current group
                group_tmp.append((token, label))
        elif label == 'O':
            # An 'O' Tag is always outside. Thus if we scan one, it means that the current group is over
            if group_tmp:
                groups.append((entity, group_tmp))
                group_tmp = [] # New Group will begin
            group_started = False
        else:
            print(f"Filtered faulty classification: ({token}, {label})")
    
    # Flush last word
    if group_tmp:
        groups.append((entity, group_tmp))

    return groups
    # for group in groups:
    #     _, entity = group[1][0]
    #     entity = entity[2:] # Get rid of  "B-" tag
    #     tokens = [item[0] for item in group]
    #     group_joined = smart_join(tokens)
    #     print(f"\tGroup: {entity}, Text: {group_joined}")

    # print("\n")

def 

def process_speech(classified_paragraphs):
    for index, p in enumerate(classified_paragraphs):
        groups = extract_groups(index, p)
        print(groups)



In [30]:
process_speech(classified_paragraphs)

Processing paragraph 0
[('EPPOL', [('Frau', 'B-EPPOL'), ('Präsidentin', 'I-EPPOL')]), ('EPPOL', [('unser', 'B-EPPOL'), ('Finanzminister', 'I-EPPOL'), ('Olaf', 'I-EPPOL'), ('Scho', 'I-EPPOL'), ('##l', 'I-EPPOL'), ('##z', 'I-EPPOL')]), ('EOPOL', [('die', 'B-EOPOL'), ('Bundesregierung', 'I-EOPOL')]), ('EOPOL', [('den', 'B-EOPOL'), ('Deutschen', 'I-EOPOL'), ('Bundestag', 'I-EOPOL')]), ('PGEN', [('aller', 'B-PGEN'), ('Menschen', 'I-PGEN'), ('in', 'I-PGEN'), ('unserem', 'I-PGEN'), ('Land', 'I-PGEN')])]
Processing paragraph 1
[('EPPOL', [('der', 'B-EPPOL'), ('Umwelt', 'I-EPPOL'), ('##ministerin', 'I-EPPOL')]), ('EOPOL', [('die', 'B-EOPOL'), ('gesamte', 'I-EOPOL'), ('Bundesregierung', 'I-EOPOL'), (',', 'I-EOPOL'), ('also', 'I-EOPOL'), ('alle', 'I-EOPOL'), ('Minister', 'I-EOPOL'), ('##ien', 'I-EOPOL')]), ('EOPOL', [('alle', 'B-EOPOL'), ('Minister', 'I-EOPOL'), ('##ien', 'I-EOPOL')]), ('EOPOL', [('des', 'B-EOPOL'), ('Verkehrs', 'I-EOPOL'), ('##ministeriums', 'I-EOPOL')])]
Processing paragraph 2


In [13]:
print(classified_paragraphs[8])

[('[CLS]', 'O'), ('Umweltschutz', 'O'), ('wird', 'O'), ('auch', 'O'), ('hier', 'O'), ('immer', 'O'), ('wieder', 'O'), ('als', 'O'), ('Streit', 'O'), ('##the', 'O'), ('##ma', 'O'), ('wahrgenommen', 'O'), ('.', 'O'), ('Das', 'O'), ('ist', 'O'), ('gut', 'O'), ('so', 'O'), (';', 'O'), ('denn', 'O'), ('wir', 'O'), ('wollen', 'O'), ('den', 'O'), ('Wett', 'O'), ('##streit', 'O'), ('um', 'O'), ('die', 'O'), ('besten', 'O'), ('politischen', 'O'), ('Konzepte', 'O'), ('und', 'O'), ('Ideen', 'O'), ('.', 'O'), ('Was', 'O'), ('aber', 'O'), ('untersch', 'O'), ('##ätzt', 'O'), ('wird', 'O'), (',', 'O'), ('ist', 'O'), (',', 'O'), ('wie', 'O'), ('verbinden', 'O'), ('##d', 'O'), ('die', 'O'), ('Wirkung', 'O'), ('des', 'O'), ('Umweltschutz', 'O'), ('##es', 'O'), ('ist', 'O'), ('.', 'O'), ('Das', 'B-EOPOL'), ('Grüne', 'I-EOPOL'), ('Band', 'I-EOPOL'), ('ist', 'O'), ('wirklich', 'O'), ('ein', 'O'), ('Symbol', 'O'), ('dafür', 'O'), ('.', 'O'), ('Wir', 'O'), ('verdanken', 'O'), ('der', 'O'), ('Umwelt', 'I-EOMO